In [1]:
!pip install gurobipy pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00


In [2]:
from pyomo.environ import *

# Problema 1

In [3]:
# Creando modelo base
problem_1 = ConcreteModel()

# define sets
problem_1.I = RangeSet(1, 2)
problem_1.J = RangeSet(1, 2)
problem_1.K = RangeSet(1, 3)

# define data
flight_cost = { (1,1): 120000, (1,2): 80000, (2,1): 90000, (2,2): 60000}
capacity = { (1,1): 20, (1,2): 4, (1,3): 3, (2,1): 15, (2,2): 1, (2,3): 2}
demand = { 1: 570, 2: 25, 3: 18}
demandB = 350
airport_capacity = { 1: 2000, 2: 150, 3: 230}

# define parameters
problem_1.c = Param(problem_1.I, problem_1.J, initialize=flight_cost)
problem_1.a = Param(problem_1.I, problem_1.K, initialize=capacity)
problem_1.demA = Param(problem_1.K, initialize=demand)
problem_1.demB = Param(initialize=demandB)
problem_1.airC = Param(problem_1.K, initialize=airport_capacity)

# define variables
problem_1.x = Var(problem_1.I, problem_1.J, domain=NonNegativeIntegers)

# define objective function
problem_1.z = Objective(expr=sum(problem_1.c[i,j] * problem_1.x[i, j] for i in problem_1.I for j in problem_1.J), sense=minimize)

# define constraints
problem_1.r1 = ConstraintList()
for k in problem_1.K:
    problem_1.r1.add(sum(problem_1.a[i,k] * problem_1.x[i,1] for i in problem_1.I) >= problem_1.demA[k])
problem_1.r2 = Constraint(expr=sum(problem_1.a[i,1] * problem_1.x[i,2] for i in problem_1.I) >= problem_1.demB)
problem_1.r3 = Constraint(expr=sum(problem_1.x[i,2] * problem_1.a[i,2] * 1/120 for i in problem_1.I) + sum(problem_1.x[i,2] * problem_1.a[i,3] * 1/90 for i in problem_1.I) <= 1)
problem_1.r4 = Constraint(expr=sum(problem_1.x[1,j] * 1/120 for j in problem_1.J) + sum(problem_1.x[2,j] * 1/85 for j in problem_1.J) <= 1)
problem_1.r5 = ConstraintList()
for k in problem_1.K:
    problem_1.r5.add(sum(problem_1.a[i,k]*problem_1.x[i,j] for i in problem_1.I for j in problem_1.J) <= problem_1.airC[k])

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_1)

# Imprimiendo resultados
print('Objective function value: ', problem_1.z())
for i in problem_1.I:
    for j in problem_1.J:
        print('x[',i,',',j,'] = ', problem_1.x[i,j]())
print('z = ', value(problem_1.z))

Objective function value:  4820000.0
x[ 1 , 1 ] =  21.0
x[ 1 , 2 ] =  10.0
x[ 2 , 1 ] =  10.0
x[ 2 , 2 ] =  10.0
z =  4820000.0


# Problema 2

In [4]:
problem_2 = ConcreteModel()

# define sets
problem_2.I = RangeSet(1, 3)
problem_2.J = RangeSet(1, 2)

# define data
loss = { 1: 62, 2: 1, 3: 30}
cost = { 1: 40, 2: 30}
items_per_day = 350
amount_per_cut = { (1,1): 2, (1,2): 0, (2,1): 1, (2,2): 2, (3,1): 0, (3,2): 3}
waste_cost = 1

# define parameters
problem_2.D = Param(problem_2.I, initialize=loss)
problem_2.C = Param(problem_2.J, initialize=cost)
problem_2.B = Param(initialize=items_per_day)
problem_2.A = Param(problem_2.I, problem_2.J, initialize=amount_per_cut)
problem_2.Wcost = Param(initialize=waste_cost)

# define variables
problem_2.Inv = Var(problem_2.J, domain=NonNegativeIntegers)
problem_2.x = Var(problem_2.I, domain=NonNegativeIntegers)

# define objective function
problem_2.z = Objective(expr=sum(problem_2.D[i] * problem_2.x[i] * problem_2.Wcost for i in problem_2.I) + sum(problem_2.C[j] * problem_2.Inv[j] for j in problem_2.J), sense=minimize)

# define constraints
problem_2.r1 = ConstraintList()
for j in problem_2.J:
    problem_2.r1.add(sum(problem_2.A[i,j] * problem_2.x[i] for i in problem_2.I) >= problem_2.B)
problem_2.r2 = ConstraintList()
for j in problem_2.J:
    problem_2.r2.add(problem_2.Inv[j] == sum(problem_2.A[i,j] * problem_2.x[i] for i in problem_2.I) - problem_2.B)

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_2)

# printing results
print('Objective function value: ', problem_2.z())
for i in problem_2.I:
    print('x[',i,'] = ', problem_2.x[i]())

Objective function value:  5630.0
x[ 1 ] =  87.0
x[ 2 ] =  176.0
x[ 3 ] =  -0.0


# Problema 3

In [5]:
problem_3 = ConcreteModel()

# define sets
problem_3.T = RangeSet(1, 4)

# demand dictionary for dem{T}
demand = { 1: 2800, 2: 2200, 3: 3200, 4: 2500}
inventory_cost = 2
overtime_cost = 10
prod_capacity = 2700
overtime_capacity = 300

# define parameters
problem_3.dem = Param(problem_3.T, initialize=demand)
problem_3.h = Param(initialize=inventory_cost)
problem_3.ce = Param(initialize=overtime_cost)
problem_3.cap = Param(initialize=prod_capacity)
problem_3.capE = Param(initialize=overtime_capacity)

# define variables
problem_3.x = Var(problem_3.T, domain=NonNegativeIntegers)
problem_3.e = Var(problem_3.T, domain=NonNegativeIntegers)
problem_3.I = Var(problem_3.T.union({0}), domain=NonNegativeIntegers)

# define objective function
problem_3.z = Objective(expr=sum(problem_3.e[t] * problem_3.ce + problem_3.I[t] * problem_3.h for t in problem_3.T), sense=minimize)

# define constraints
problem_3.r1 = ConstraintList()
for t in problem_3.T:
    problem_3.r1.add(problem_3.x[t] <= problem_3.cap)
problem_3.r2 = ConstraintList()
for t in problem_3.T:
    problem_3.r2.add(problem_3.e[t] <= problem_3.capE)
problem_3.r3 = ConstraintList()
for t in problem_3.T:
    problem_3.r3.add(problem_3.I[t-1] + problem_3.x[t] + problem_3.e[t] == problem_3.I[t] + problem_3.dem[t])
problem_3.r4 = Constraint(expr=problem_3.I[0] == 0)
problem_3.r5 = Constraint(expr=problem_3.I[4] == 0)

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_3)

# print results
print('Objective function value: ', problem_3.z())
# print matrix demand, x, e, i

for t in problem_3.T:
    print('x[',t,'] = ', problem_3.x[t]())
    print('e[',t,'] = ', problem_3.e[t]())
    print('I[',t,'] = ', problem_3.I[t]())




Objective function value:  2000.0
x[ 1 ] =  2700.0
e[ 1 ] =  100.0
I[ 1 ] =  0.0
x[ 2 ] =  2700.0
e[ 2 ] =  0.0
I[ 2 ] =  500.0
x[ 3 ] =  2700.0
e[ 3 ] =  0.0
I[ 3 ] =  -0.0
x[ 4 ] =  2500.0
e[ 4 ] =  -0.0
I[ 4 ] =  0.0


## Variación propuesta


# Problema 4

In [6]:
problem_4 = ConcreteModel()

# define sets
problem_4.I = RangeSet(1, 4)
problem_4.J = RangeSet(1, 3)

# define data
utility = { 1: 250, 2: 280, 3: 500, 4: 360}
max_weight_capacity = { 1: 16, 2: 20, 3: 14}
max_volume_capacity = { 1: 200, 2: 250, 3: 150}
load_qty = { 1: 20, 2: 15, 3: 8, 4: 10}
max_volume_of_type = { 1: 1, 2: 2, 3: 10, 4: 6}

# define parameters
problem_4.u = Param(problem_4.I, initialize=utility)
problem_4.P = Param(problem_4.J, initialize=max_weight_capacity)
problem_4.V = Param(problem_4.J, initialize=max_volume_capacity)
problem_4.C = Param(problem_4.I, initialize=load_qty)
problem_4.Vol = Param(problem_4.I, initialize=max_volume_of_type)

# define variables
problem_4.x = Var(problem_4.I, problem_4.J, domain=NonNegativeReals)
problem_4.G = Var(domain=NonNegativeReals)

# define objective function
problem_4.z = Objective(expr=sum(problem_4.u[i] * problem_4.x[i,j] for i in problem_4.I for j in problem_4.J), sense=maximize)

# define constraints
problem_4.r1 = ConstraintList()
for j in problem_4.J:
    problem_4.r1.add(sum(problem_4.x[i,j] for i in problem_4.I) <= problem_4.P[j])
problem_4.r2 = ConstraintList()
for j in problem_4.J:
    problem_4.r2.add(sum(problem_4.Vol[i] * problem_4.x[i,j] for i in problem_4.I) <= problem_4.V[j])
problem_4.r3 = ConstraintList()
for i in problem_4.I:
    problem_4.r3.add(sum(problem_4.x[i,j] for j in problem_4.J) <= problem_4.C[i])
problem_4.r4 = ConstraintList()
for j in problem_4.J:
    problem_4.r4.add(sum(problem_4.x[i,j]/problem_4.P[j] for i in problem_4.I) == problem_4.G)

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_4)

# print results
print('Objective function value: ', problem_4.z())
for i in problem_4.I:
    for j in problem_4.J:
        print('x[',i,',',j,'] = ', problem_4.x[i,j]())


Objective function value:  16050.0
x[ 1 , 1 ] =  8.0
x[ 1 , 2 ] =  0.0
x[ 1 , 3 ] =  9.0
x[ 2 , 1 ] =  0.0
x[ 2 , 2 ] =  15.0
x[ 2 , 3 ] =  0.0
x[ 3 , 1 ] =  8.0
x[ 3 , 2 ] =  0.0
x[ 3 , 3 ] =  0.0
x[ 4 , 1 ] =  0.0
x[ 4 , 2 ] =  5.0
x[ 4 , 3 ] =  5.0


# Problema 5

In [7]:
problem_5 = ConcreteModel()

# define sets
problem_5.I = RangeSet(1, 3)
problem_5.J = RangeSet(1, 3)
problem_5.K = RangeSet(1, 3)

# define data
composition = { (1,1): 80, (1,2): 10, (1,3): 5, (2,1): 45, (2,2): 30, (2,3): 20, (3,1): 30, (3,2): 40, (3,3): 25}
required_composition_in_gas = { (1,1): 60, (1,2): 25, (1,3): 10, (2,1): 30, (2,2): 20, (2,3): 15, (3,1): 40, (3,2): 35, (3,3): 20}
cost_per_barrel_of_crude = { 1: 250, 2: 120, 3: 200}
budget = 50000
available_crude = { 2: 300, 3: 700}
A_barrels_to_buy = 10
demand = { 1: 60, 2: 70}


# define parameters
problem_5.PC = Param(problem_5.J, problem_5.K, initialize=composition)
problem_5.PG = Param(problem_5.I, problem_5.K, initialize=required_composition_in_gas)
problem_5.CostoC = Param(problem_5.J, initialize=cost_per_barrel_of_crude)
problem_5.Presu = Param(initialize=budget)
problem_5.available_crude = Param(RangeSet(2,3), initialize=available_crude)
problem_5.BA = Param(initialize=A_barrels_to_buy)
problem_5.DEM = Param(RangeSet(1,2), initialize=demand)

# Define variables
problem_5.X = Var(problem_5.I, problem_5.J, within=NonNegativeReals)

# Define objective function
problem_5.z = Objective(expr=sum(problem_5.X[3, j] for j in problem_5.J), sense=maximize)

# Define constraints
problem_5.r1 = ConstraintList()
for k in [1, 3]:
    problem_5.r1.add(sum(problem_5.PC[j, k] * problem_5.X[1, j] for j in problem_5.J) >= problem_5.PG[1, k] * sum(problem_5.X[1, j] for j in problem_5.J))
problem_5.r2 = Constraint(expr=sum(problem_5.PC[j, 2] * problem_5.X[1, j] for j in problem_5.J) <= problem_5.PG[1, 2] * sum(problem_5.X[1, j] for j in problem_5.J))
problem_5.r3 = ConstraintList()
for k in [2, 3]:
    problem_5.r3.add(sum(problem_5.PC[j, k] * problem_5.X[2, j] for j in problem_5.J) <= problem_5.PG[2, k] * sum(problem_5.X[2, j] for j in problem_5.J))
problem_5.r4 = Constraint(expr=sum(problem_5.PC[j, 1] * problem_5.X[2, j] for j in problem_5.J) >= problem_5.PG[2, 1] * sum(problem_5.X[2, j] for j in problem_5.J))
problem_5.r5 = ConstraintList()
for k in [2, 3]:
    problem_5.r5.add(sum(problem_5.PC[j, k] * problem_5.X[3, j] for j in problem_5.J) >= problem_5.PG[3, k] * sum(problem_5.X[3, j] for j in problem_5.J))
problem_5.r6 = Constraint(expr=sum(problem_5.PC[j, 1] * problem_5.X[3, j] for j in problem_5.J) <= problem_5.PG[3, 1] * sum(problem_5.X[3, j] for j in problem_5.J))
problem_5.r7 = Constraint(expr=sum(problem_5.X[i, j] * problem_5.CostoC[j] for i in problem_5.I for j in problem_5.J) <= problem_5.Presu)
problem_5.r8 = Constraint(expr=sum(problem_5.X[i, 1] for i in problem_5.I) >= problem_5.BA)
problem_5.r9 = ConstraintList()
for i in [1, 2]:
    problem_5.r9.add(sum(problem_5.X[i, j] for j in problem_5.J) >= problem_5.DEM[i])

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_5)

# Print results
print("Objective Value:", problem_5.z())
print("X:")
for i in problem_5.I:
    for j in problem_5.J:
        print(i, j, problem_5.X[i, j].value)

Objective Value: 165.6696428571429
X:
1 1 25.71428571428571
1 2 34.285714285714285
1 3 0.0
2 1 35.0
2 2 35.0
2 3 0.0
3 1 0.0
3 2 82.83482142857144
3 3 82.83482142857144


## variación propuesta

Se debe agregar una restricción indicando que la mezcla estará sujeta a un 95%

In [8]:
problem_5 = ConcreteModel()

# define sets
problem_5.I = RangeSet(1, 3)
problem_5.J = RangeSet(1, 3)
problem_5.K = RangeSet(1, 3)

# define data
composition = { (1,1): 80, (1,2): 10, (1,3): 5, (2,1): 45, (2,2): 30, (2,3): 20, (3,1): 30, (3,2): 40, (3,3): 25}
required_composition_in_gas = { (1,1): 60, (1,2): 25, (1,3): 10, (2,1): 30, (2,2): 20, (2,3): 15, (3,1): 40, (3,2): 35, (3,3): 20}
cost_per_barrel_of_crude = { 1: 250, 2: 120, 3: 200}
budget = 50000
available_crude = { 2: 300, 3: 700}
A_barrels_to_buy = 10
demand = { 1: 60, 2: 70}
loss = 0.05

# define parameters
problem_5.PC = Param(problem_5.J, problem_5.K, initialize=composition)
problem_5.PG = Param(problem_5.I, problem_5.K, initialize=required_composition_in_gas)
problem_5.CostoC = Param(problem_5.J, initialize=cost_per_barrel_of_crude)
problem_5.Presu = Param(initialize=budget)
problem_5.available_crude = Param(RangeSet(2,3), initialize=available_crude)
problem_5.BA = Param(initialize=A_barrels_to_buy)
problem_5.DEM = Param(RangeSet(1,2), initialize=demand)
problem_5.LOSS = Param(initialize=loss)

# Define variables
problem_5.X = Var(problem_5.I, problem_5.J, within=NonNegativeReals)

# Define objective function
problem_5.z = Objective(expr=sum(problem_5.X[3, j] for j in problem_5.J), sense=maximize)

# Define constraints
problem_5.r1 = ConstraintList()
for k in [1, 3]:
    problem_5.r1.add(sum(problem_5.PC[j, k] * problem_5.X[1, j] for j in problem_5.J) >= problem_5.PG[1, k] * sum(problem_5.X[1, j] for j in problem_5.J))
problem_5.r2 = Constraint(expr=sum(problem_5.PC[j, 2] * problem_5.X[1, j] for j in problem_5.J) <= problem_5.PG[1, 2] * sum(problem_5.X[1, j] for j in problem_5.J))
problem_5.r3 = ConstraintList()
for k in [2, 3]:
    problem_5.r3.add(sum(problem_5.PC[j, k] * problem_5.X[2, j] for j in problem_5.J) <= problem_5.PG[2, k] * sum(problem_5.X[2, j] for j in problem_5.J))
problem_5.r4 = Constraint(expr=sum(problem_5.PC[j, 1] * problem_5.X[2, j] for j in problem_5.J) >= problem_5.PG[2, 1] * sum(problem_5.X[2, j] for j in problem_5.J))
problem_5.r5 = ConstraintList()
for k in [2, 3]:
    problem_5.r5.add(sum(problem_5.PC[j, k] * problem_5.X[3, j] for j in problem_5.J) >= problem_5.PG[3, k] * sum(problem_5.X[3, j] for j in problem_5.J))
problem_5.r6 = Constraint(expr=sum(problem_5.PC[j, 1] * problem_5.X[3, j] for j in problem_5.J) <= problem_5.PG[3, 1] * sum(problem_5.X[3, j] for j in problem_5.J))
problem_5.r7 = Constraint(expr=sum(problem_5.X[i, j] * problem_5.CostoC[j] for i in problem_5.I for j in problem_5.J) <= problem_5.Presu)
problem_5.r8 = Constraint(expr=sum(problem_5.X[i, 1] for i in problem_5.I) >= problem_5.BA)
problem_5.r9 = ConstraintList()
for i in [1, 2]:
    problem_5.r9.add(sum(problem_5.X[i, j] * (1 - problem_5.LOSS) for j in problem_5.J) >= problem_5.DEM[i])


# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_5)

# Print results
print("Objective Value:", problem_5.z())
print("X:")
for i in problem_5.I:
    for j in problem_5.J:
        print(i, j, problem_5.X[i, j].value)

Objective Value: 157.94172932330827
X:
1 1 27.06766917293233
1 2 36.090225563909776
1 3 0.0
2 1 36.8421052631579
2 2 36.8421052631579
2 3 0.0
3 1 0.0
3 2 78.97086466165413
3 3 78.97086466165413


# Problema 6

In [9]:
problem_6 = ConcreteModel()

# define sets
problem_6.I = RangeSet(1,4)
problem_6.J = RangeSet(1,4)

# define data
fixed_cost = { 3: 1000, 4: 900}
transportation_cost = { (1,1): 6, (1,2): 4, (1,3): 2, (1,4): 6,
                        (2,1): 2, (2,2): 3, (2,3): 7, (2,4): 5,
                        (3,1): 3, (3,2): 2, (3,3): 4, (3,4): 8,
                        (4,1): 6, (4,2): 3, (4,3): 4, (4,4): 2}
warehouse_demand = { 1: 700*1.25, 2: 800*1.25, 3: 500*1.25, 4: 400*1.25}
warehouse_capacity = { 1: 900, 2: 1500, 3: 900, 4: 900}

# define parameters
problem_6.CF = Param(RangeSet(3,4), initialize=fixed_cost)
problem_6.CT = Param(problem_6.I, problem_6.J, initialize=transportation_cost)
problem_6.DEM = Param(problem_6.J, initialize=warehouse_demand)
problem_6.CAP = Param(problem_6.I, initialize=warehouse_capacity)

# define variables
problem_6.x = Var(problem_6.I, problem_6.J, domain=NonNegativeIntegers)
problem_6.y = Var(RangeSet(3,4), domain=Binary)

# define objective function
problem_6.z = Objective(expr=sum(problem_6.CT[i,j] * problem_6.x[i,j] for i in problem_6.I for j in problem_6.J) + sum(problem_6.CF[i] * problem_6.y[i] for i in RangeSet(3,4)), sense=minimize)

# define constraints
problem_6.r1 = ConstraintList()
for j in problem_6.J:
    problem_6.r1.add(sum(problem_6.x[i,j] for i in problem_6.I) >= problem_6.DEM[j])
problem_6.r2 = ConstraintList()
for i in {1,2}:
    problem_6.r2.add(sum(problem_6.x[i,j] for j in problem_6.J) <= problem_6.CAP[i])
problem_6.r3 = ConstraintList()
for i in {3,4}:
    problem_6.r3.add(sum(problem_6.x[i,j] for j in problem_6.J) <= problem_6.CAP[i] * problem_6.y[i])
problem_6.r4 = Constraint(expr=sum(problem_6.y[i] for i in RangeSet(3,4)) == 1)

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_6)

# print results
print('Solución optima:')
print('Valor óptimo:', value(problem_6.z))
print('bodega, planta')
for j in problem_6.J:
    for i in problem_6.I:
        print(i, j, problem_6.x[i, j].value)

Solución optima:
Valor óptimo: 7900.0
bodega, planta
1 1 -0.0
2 1 875.0
3 1 -0.0
4 1 -0.0
1 2 -0.0
2 2 625.0
3 2 -0.0
4 2 375.0
1 3 625.0
2 3 -0.0
3 3 -0.0
4 3 -0.0
1 4 -0.0
2 4 -0.0
3 4 -0.0
4 4 500.0


# variación del estudiante

considerar que las nuevas plantas tienen una capacidad desconocida

# Problema 7

In [10]:
problem_7 = ConcreteModel()

# define sets
problem_7.I = RangeSet(1,5)
problem_7.J = RangeSet(1,3)
problem_7.T = RangeSet(1,3)

# define variables
variable_cost = { (1,1): 250, (1,2): 125, (1,3): 200,
                (2,1): 125, (2,2): 300, (2,3): 350,
                (3,1): 500, (3,2): 450, (3,3): 600,
                (4,1): 100, (4,2): 200, (4,3): 125,
                (5,1): 250, (5,2): 400, (5,3): 300}

fixed_cost = { (1,1): 600, (1,2): 700, (1,3): 800,
                (2,1): 200, (2,2): 150, (2,3): 300,
                (3,1): 800, (3,2): 1000, (3,3): 900,
                (4,1): 300, (4,2): 120, (4,3): 250,
                (5,1): 500, (5,2): 400, (5,3): 225}

extraction_capacity = { (1,1): 600, (1,2): 200, (1,3): 800,
                        (2,1): 900, (2,2): 300, (2,3): 500,
                        (3,1): 400, (3,2): 800, (3,3): 300}

demand = { (1,1): 50, (1,2): 75, (1,3): 80,
            (2,1): 25, (2,2): 8, (2,3): 17,
            (3,1): 9, (3,2): 10, (3,3): 5,
            (4,1): 70, (4,2): 85, (4,3): 52,
            (5,1): 60, (5,2): 26, (5,3): 35}

# define parameters
problem_7.CF = Param(problem_7.I, problem_7.J, initialize=variable_cost)
problem_7.CT = Param(problem_7.I, problem_7.J, initialize=fixed_cost)
problem_7.Cap = Param(problem_7.J, problem_7.T, initialize=extraction_capacity)
problem_7.Dem = Param(problem_7.I, problem_7.T, initialize=demand)
problem_7.M = Param(initialize=1000000)

# define variables
problem_7.x = Var(problem_7.I, problem_7.J, problem_7.T, domain=NonNegativeReals)
problem_7.y = Var(problem_7.I, problem_7.J, domain=Binary)

# define objective function
problem_7.z = Objective(expr=sum(problem_7.CF[i,j] * problem_7.x[i,j,t] for i in problem_7.I for j in problem_7.J for t in problem_7.T) + sum(problem_7.CT[i,j] * problem_7.y[i,j] for i in problem_7.I for j in problem_7.J), sense=minimize)

# define constraints
problem_7.r1 = ConstraintList()
for j in problem_7.J:
    for t in problem_7.T:
        problem_7.r1.add(sum(problem_7.x[i,j,t] for i in problem_7.I) <= problem_7.Cap[j,t])
problem_7.r2 = ConstraintList()
for i in problem_7.I:
    for t in problem_7.T:
        problem_7.r1.add(sum(problem_7.x[i,j,t] for j in problem_7.J) >= problem_7.Dem[i,t])
problem_7.r3 = ConstraintList()
for i in problem_7.I:
    for j in problem_7.J:
        problem_7.r1.add(sum(problem_7.x[i,j,t] for t in problem_7.T) <= problem_7.M * problem_7.y[i,j])

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_7)

print('Solución optima:')
print('mineral, mina, cuatrimestre')
for j in problem_7.J:
    for i in problem_7.I:
        for t in problem_7.T:
            print(i, j, t, problem_7.x[i,j,t].value)
print('Valor óptimo:', value(problem_7.z))

Solución optima:
mineral, mina, cuatrimestre
1 1 1 0.0
1 1 2 0.0
1 1 3 0.0
2 1 1 25.0
2 1 2 8.0
2 1 3 17.0
3 1 1 0.0
3 1 2 0.0
3 1 3 0.0
4 1 1 70.0
4 1 2 85.0
4 1 3 52.0
5 1 1 60.0
5 1 2 26.0
5 1 3 35.0
1 2 1 50.0
1 2 2 75.0
1 2 3 80.0
2 2 1 0.0
2 2 2 0.0
2 2 3 0.0
3 2 1 9.0
3 2 2 10.0
3 2 3 5.0
4 2 1 0.0
4 2 2 0.0
4 2 3 0.0
5 2 1 0.0
5 2 2 0.0
5 2 3 0.0
1 3 1 0.0
1 3 2 0.0
1 3 3 0.0
2 3 1 0.0
2 3 2 0.0
2 3 3 0.0
3 3 1 0.0
3 3 2 0.0
3 3 3 0.0
4 3 1 0.0
4 3 2 0.0
4 3 3 0.0
5 3 1 0.0
5 3 2 0.0
5 3 3 0.0
Valor óptimo: 96325.0


# Problema 8

In [11]:
problem_8 = ConcreteModel()

# define sets
problem_8.I = RangeSet(0,7)
problem_8.A = Set(within=problem_8.I * problem_8.I, initialize=lambda model: [(i,j) for i in model.I for j in model.I if i != j])

# define data
cost = {
    (0, 1): 3, (0, 2): 6, (0, 3): 1000, (0, 4): 10, (0, 5): 1000, (0, 6): 1000, (0, 7): 1000,
    (1, 0): 8, (1, 2): 10, (1, 3): 2, (1, 4): 1, (1, 5): 9, (1, 6): 3, (1, 7): 1000,
    (2, 0): 7, (2, 1): 6, (2, 3): 1, (2, 4): 5, (2, 5): 3, (2, 6): 5, (2, 7): 1000,
    (3, 0): 1000, (3, 1): 8, (3, 2): 8, (3, 4): 3, (3, 5): 9, (3, 6): 10, (3, 7): 1000,
    (4, 0): 4, (4, 1): 7, (4, 2): 7, (4, 3): 4, (4, 5): 2, (4, 6): 5, (4, 7): 10,
    (5, 0): 1000, (5, 1): 4, (5, 2): 2, (5, 3): 9, (5, 4): 1, (5, 6): 2, (5, 7): 8,
    (6, 0): 1000, (6, 1): 6, (6, 2): 2, (6, 3): 7, (6, 4): 10, (6, 5): 2, (6, 7): 3,
    (7, 0): 1000, (7, 1): 1000, (7, 2): 1000, (7, 3): 1000, (7, 4): 1, (7, 5): 6, (7, 6): 6
}
door = {0: 0, 1: 6, 2: 8, 3: 8, 4: 2, 5: 7, 6: 6, 7: 3}

# defina parameters
problem_8.c = Param(problem_8.A, initialize=cost)
problem_8.door = Param(problem_8.I, initialize=door)

# define variables
problem_8.x = Var(problem_8.A, domain=Binary)
problem_8.y = Var(problem_8.I, domain=Binary)

# define objective function
problem_8.z = Objective(expr=sum(problem_8.c[i,j] * problem_8.x[i,j] for (i,j) in problem_8.A if i!=j) + sum(problem_8.door[i] * problem_8.y[i] for i in problem_8.I), sense=minimize)

# define constraints
problem_8.r1 = Constraint(expr=sum(problem_8.x[0,i] for i in problem_8.I if i != 0) == 1)
problem_8.r2 = Constraint(expr=sum(problem_8.x[i,7] for i in problem_8.I if i != 7) == 1)
problem_8.r3 = ConstraintList()
for j in problem_8.I:
    if j != 0 and j != 7:
        problem_8.r3.add(sum(problem_8.x[i,j] for i in problem_8.I if i != j) == sum(problem_8.x[j,i] for i in problem_8.I if i != j))
problem_8.r4 = ConstraintList()
for i in problem_8.I:
    problem_8.r4.add(problem_8.y[i] >= sum(problem_8.x[j,i] for j in problem_8.I if j != i))

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_8)

# print results
print('Solución optima:')
for x in problem_8.x:
    if value(problem_8.x[x]) > 0:
        print(x, value(problem_8.x[x]))
print('Valor óptimo:', value(problem_8.z))

Solución optima:
(0, 1) 1.0
(1, 6) 1.0
(6, 7) 1.0
Valor óptimo: 24.0


# Problema 9

In [12]:
problem_9 = ConcreteModel()

# define set T 1..5
problem_9.T = RangeSet(1,5)

demand = {1: 100, 2: 200, 3: 300, 4: 400, 5: 200}
storage_cost = 10
miss_cost = 30
salary = 100
firing_cost = 100
doors_per_worker_week = 8

# define parameters
problem_9.D = Param(problem_9.T, initialize=demand)
problem_9.Ch = Param(initialize=storage_cost)
problem_9.Cf = Param(initialize=miss_cost)
problem_9.C1 = Param(initialize=salary)
problem_9.C2 = Param(initialize=firing_cost)
problem_9.Num = Param(initialize=doors_per_worker_week)

# define variables
problem_9.x = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.I = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.y = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.z = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.m = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define objective function
problem_9.obj = Objective(expr=sum(problem_9.C1 * (problem_9.y[t] + problem_9.z[t] + (problem_9.z[t]/5)) + problem_9.C2 * problem_9.m[t] + problem_9.Ch * problem_9.I[t] + problem_9.Cf * problem_9.x[t] for t in problem_9.T), sense=minimize)

# define constraints
problem_9.r1 = ConstraintList()
for t in problem_9.T:
    problem_9.r1.add(problem_9.I[t-1] + problem_9.y[t] * problem_9.Num == problem_9.D[t] + problem_9.x[t-1] + problem_9.I[t] - problem_9.x[t])
problem_9.r2 = ConstraintList()
for t in problem_9.T:
    problem_9.r2.add(problem_9.y[t] == problem_9.y[t-1] - problem_9.m[t-1] + problem_9.z[t-1] - problem_9.z[t]/5 + problem_9.z[t-1]/5)
problem_9.r3 = Constraint(expr=problem_9.x[0] == 0)
problem_9.r4 = Constraint(expr=problem_9.I[0] == 10)
problem_9.r5 = Constraint(expr=problem_9.y[0] == 20)
problem_9.r6 = Constraint(expr=problem_9.z[0] == 0)
problem_9.r7 = Constraint(expr=problem_9.m[0] == 0)

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_9)

print('Solución optima:')
# print as matrix
for t in problem_9.T:
    print(t, sep=' ', end=' ')
print('')
print('Puertas no entregadas', end=' ')
for x in problem_9.x:
    print(value(problem_9.x[x]), sep=' ', end=' ')
print('')
print('Puertas en inventario', end=' ')
for i in problem_9.I:
    print(value(problem_9.I[i]), sep=' ', end=' ')
print('')
print('Expertos en producción', end=' ')
for y in problem_9.y:
    print(value(problem_9.y[y]), sep=' ', end=' ')
print('')
print('Novatos en capacitación', end=' ')
for z in problem_9.z:
    print(value(problem_9.z[z]), sep=' ', end=' ')
print('')
print('Operarios despedidos', end=' ')
for m in problem_9.m:
    print(value(problem_9.m[m]), sep=' ', end=' ')
print('')
print('Valor óptimo:', value(problem_9.obj))


Solución optima:
1 2 3 4 5 
Puertas no entregadas -0.0 -0.0 -0.0 54.0 -0.0 0.0 
Puertas en inventario 54.0 86.0 66.0 -0.0 2.0 10.0 
Expertos en producción 18.0 29.0 35.0 35.0 32.0 20.0 
Novatos en capacitación 10.0 5.0 0.0 0.0 0.0 0.0 
Operarios despedidos 0.0 -0.0 0.0 3.0 -0.0 0.0 
Valor óptimo: 20700.0


## Variación propuesta

In [13]:
problem_9 = ConcreteModel()

# define set T 1..5
problem_9.T = RangeSet(1,5)

demand = {1: 100, 2: 200, 3: 300, 4: 400, 5: 200}
storage_cost = 10
miss_cost = 30
salary = 100
firing_cost = 100
doors_per_worker_week = 8
beginners_training_days = 2

# define parameters
problem_9.D = Param(problem_9.T, initialize=demand)
problem_9.Ch = Param(initialize=storage_cost)
problem_9.Cf = Param(initialize=miss_cost)
problem_9.C1 = Param(initialize=salary)
problem_9.C2 = Param(initialize=firing_cost)
problem_9.Num = Param(initialize=doors_per_worker_week)
problem_9.training_days = Param(initialize=beginners_training_days)

# define variables
problem_9.x = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.I = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.y = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.z = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)
problem_9.m = Var(problem_9.T.union({0}), domain=NonNegativeIntegers)

# define objective function
problem_9.obj = Objective(expr=sum(problem_9.C1 * (problem_9.y[t] + problem_9.z[t] + (problem_9.z[t]/problem_9.training_days)) + problem_9.C2 * problem_9.m[t] + problem_9.Ch * problem_9.I[t] + problem_9.Cf * problem_9.x[t] for t in problem_9.T), sense=minimize)

# define constraints
problem_9.r1 = ConstraintList()
for t in problem_9.T:
    problem_9.r1.add(problem_9.I[t-1] + problem_9.y[t] * problem_9.Num == problem_9.D[t] + problem_9.x[t-1] + problem_9.I[t] - problem_9.x[t])
problem_9.r2 = ConstraintList()
for t in problem_9.T:
    problem_9.r2.add(
            problem_9.y[t] ==
            problem_9.y[t - 1] - problem_9.m[t - 1] + problem_9.z[t - 1] - problem_9.z[t] / problem_9.training_days
        )
problem_9.r3 = Constraint(expr=problem_9.x[0] == 0)
problem_9.r4 = Constraint(expr=problem_9.I[0] == 10)
problem_9.r5 = Constraint(expr=problem_9.y[0] == 20)
problem_9.r6 = Constraint(expr=problem_9.z[0] == 0)
problem_9.r7 = Constraint(expr=problem_9.m[0] == 0)

# set solver
solver = SolverFactory('gurobi')
results = solver.solve(problem_9)

print('Solución optima:')
# print as matrix
for t in problem_9.T:
    print(t, sep=' ', end=' ')
print('')
print('Puertas no entregadas', end=' ')
for x in problem_9.x:
    print(value(problem_9.x[x]), sep=' ', end=' ')
print('')
print('Puertas en inventario', end=' ')
for i in problem_9.I:
    print(value(problem_9.I[i]), sep=' ', end=' ')
print('')
print('Expertos en producción', end=' ')
for y in problem_9.y:
    print(value(problem_9.y[y]), sep=' ', end=' ')
print('')
print('Novatos en capacitación', end=' ')
for z in problem_9.z:
    print(value(problem_9.z[z]), sep=' ', end=' ')
print('')
print('Operarios despedidos', end=' ')
for m in problem_9.m:
    print(value(problem_9.m[m]), sep=' ', end=' ')
print('')
print('Valor óptimo:', value(problem_9.obj))


Solución optima:
1 2 3 4 5 
Puertas no entregadas 42.0 0.0 0.0 102.0 6.0 0.0 
Puertas en inventario 0.0 6.0 2.0 0.0 0.0 10.0 
Expertos en producción 6.0 31.0 37.0 37.0 37.0 20.0 
Novatos en capacitación 28.0 6.0 0.0 0.0 0.0 0.0 
Operarios despedidos 0.0 0.0 0.0 0.0 -0.0 0.0 
Valor óptimo: 24480.0


# Problema 10

In [14]:
problem_10 = ConcreteModel()

# define sets
problem_10.I = RangeSet(1,2)
problem_10.J = RangeSet(1,4)
problem_10.K = RangeSet(1,3)

# define data
max_cap = {1: 5000, 2: 3000, 3: 4000}
mper_perton_permine = {
    (1, 1): 0.15, (2, 1): 0.20, (3, 1): 0.40, (4, 1): 0.25,
    (1, 2): 0.10, (2, 2): 0.30, (3, 2): 0.10, (4, 2): 0.40,
    (1, 3): 0.5, (2, 3): 0.10, (3, 3): 0.70, (4, 3): 0.10
}
cost_ton = {1: 70, 2: 50, 3: 65}
sale_price = {1: 250, 2: 170}

# define parameters
problem_10.M = Param(initialize=10000)
problem_10.Cmax = Param(problem_10.K, initialize=max_cap)
problem_10.Por = Param(problem_10.J, problem_10.K, initialize=mper_perton_permine)
problem_10.CT = Param(problem_10.K, initialize=cost_ton)
problem_10.PV = Param(problem_10.I, initialize=sale_price)

# define variables
problem_10.X = Var(problem_10.K, problem_10.I, domain=NonNegativeReals)
problem_10.Y = Var(problem_10.K - {3}, domain=Binary)
problem_10.Z = Var(domain=Binary)

# define objective function
problem_10.obj = Objective(expr=sum(problem_10.PV[i] * problem_10.X[k,i] for i in problem_10.I for k in problem_10.K) - sum(problem_10.CT[k] * problem_10.X[k,i] for i in problem_10.I for k in problem_10.K), sense=maximize)

# define constraints
problem_10.r1 = ConstraintList()
for k in {1,2}:
    problem_10.r1.add(sum(problem_10.X[k,i] for i in problem_10.I) <= problem_10.Cmax[k] * problem_10.Y[k])
problem_10.r2 = ConstraintList()
for k in {3}:
    problem_10.r2.add(sum(problem_10.X[k,i] for i in problem_10.I) <= problem_10.Cmax[k])
problem_10.r3 = Constraint(expr=sum(problem_10.X[k,1] * problem_10.Por[1,k] for k in problem_10.K) <= sum(problem_10.X[k,1] * 0.4 for k in problem_10.K))
problem_10.r4 = Constraint(expr=sum(problem_10.X[k,1] * problem_10.Por[2,k] for k in problem_10.K) <= sum(problem_10.X[k,1] * 0.75 for k in problem_10.K))
problem_10.r5 = Constraint(expr=sum(problem_10.X[k,1] * problem_10.Por[4,k] for k in problem_10.K) >= sum(problem_10.X[k,1] * 0.3 for k in problem_10.K))
problem_10.r6 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[2,k] for k in problem_10.K) >= sum(problem_10.X[k,2] * 0.25 for k in problem_10.K))
problem_10.r7 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[2,k] for k in problem_10.K) <= sum(problem_10.X[k,2] * 0.8 for k in problem_10.K))
problem_10.r8 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[3,k] for k in problem_10.K) >= sum(problem_10.X[k,2] * 0.5 for k in problem_10.K))
problem_10.r9 = Constraint(expr=sum(problem_10.X[k,2] * problem_10.Por[4,k] for k in problem_10.K) <= sum(problem_10.X[k,2] * 0.66 for k in problem_10.K))
problem_10.r10 = Constraint(expr=problem_10.Y[1] == 1 - problem_10.Y[2])
problem_10.r11 = Constraint(expr=problem_10.X[3,1] <= problem_10.Z * problem_10.M)
problem_10.r12 = Constraint(expr=problem_10.X[3,2] <= (1 - problem_10.Z) * problem_10.M)

solver = SolverFactory('gurobi')
results = solver.solve(problem_10)

print('Solución optima:')
for x in problem_10.X:
    print(x, value(problem_10.X[x]))
print('Valor óptimo:', value(problem_10.obj))

Solución optima:
(1, 1) 0.0
(1, 2) 0.0
(2, 1) 3000.0
(2, 2) 0.0
(3, 1) 1500.0000000000007
(3, 2) 0.0
Valor óptimo: 877500.0000000002


# Problema 11

In [15]:
problem_11 = ConcreteModel()

# define sets
problem_11.I = RangeSet(1,2)
problem_11.J = RangeSet(1,3)
problem_11.K =RangeSet(1,2)

# define data
demand = {1: 3000, 2: 2000}
capacity = {(1,1,1): 0.5, (1,1,2): 0.6, (1,2,1): 3.75, (1,2,2): 4, (1,3,1): 0.6, (1,3,2): 0.65,
            (2,1,1): 0.5, (2,1,2): 0.6, (2,2,1): 3.3, (2,2,2): 3.9, (2,3,1): 0.75, (2,3,2): 0.78}
timeontask = {(1,1): 1, (1,2): 3, (1,3): 1, (2,1): 1, (2,2): 2.5, (2,3): 1.5}

# define parameters
problem_11.Dem = Param(problem_11.I, initialize=demand)
problem_11.C = Param(problem_11.I, problem_11.J, problem_11.K, initialize=capacity)
problem_11.CapH = Param(initialize=200)
problem_11.CapHE = Param(initialize=50)
problem_11.CHE = Param(initialize=9)
problem_11.T = Param(problem_11.I, problem_11.J, initialize=timeontask)

# define variables
problem_11.X = Var(problem_11.I, problem_11.J, problem_11.K, within=NonNegativeIntegers)
problem_11.H = Var(within=NonNegativeReals)

# define objective function
problem_11.obj = Objective(expr=sum(problem_11.C[i,j,k] * problem_11.X[i,j,k] for i in problem_11.I for j in problem_11.J for k in problem_11.K) + problem_11.CHE * problem_11.H, sense=minimize)

# define constraints
problem_11.r1 = ConstraintList()
for i in problem_11.I:
    for j in problem_11.J:
        problem_11.r1.add(sum(problem_11.X[i,j,k] for k in problem_11.K) >= problem_11.Dem[i])
problem_11.r2 = Constraint(expr=problem_11.H <= problem_11.CapHE)
problem_11.r3 = Constraint(expr=sum((problem_11.T[i,j]/60) * problem_11.X[i,j,1] for i in problem_11.I for j in problem_11.J) <= problem_11.H + problem_11.CapH)

solver = SolverFactory('gurobi')
results = solver.solve(problem_11)

print('Solución optima:')
for x in problem_11.X:
    print(x, value(problem_11.X[x]))
print('Valor óptimo:', value(problem_11.obj))


Solución optima:
(1, 1, 1) 3000.0
(1, 1, 2) -0.0
(1, 2, 1) 666.0
(1, 2, 2) 2334.0
(1, 3, 1) 1.0
(1, 3, 2) 2999.0
(2, 1, 1) 2000.0
(2, 1, 2) -0.0
(2, 2, 1) 2000.0
(2, 2, 2) -0.0
(2, 3, 1) -0.0
(2, 3, 2) 2000.0
Valor óptimo: 24443.45


# Problema 12

In [16]:
problem_12 = ConcreteModel()

# define sets
problem_12.I = RangeSet(1,5)
problem_12.J = RangeSet(1,4)
problem_12.K = RangeSet(1,2)

# define data
cost = {1: 50, 2: 30, 3: 35, 4: 60, 5: 40}
demand = {(1,1): 3, (1,2): 3, (2,1): 2, (2,2): 2, (3,1): 2, (3,2): 2, (4,1): 4, (4,2): 3}

# Param C{I}
problem_12.C = Param(problem_12.I, initialize=cost)
problem_12.DEM = Param(problem_12.J, problem_12.K, initialize=demand)

# define variables
problem_12.x = Var(problem_12.I, problem_12.J, problem_12.K, within=NonNegativeIntegers)

# define objective function
problem_12.z = Objective(expr=sum(problem_12.C[i] * problem_12.x[i,j,k] for i in problem_12.I for j in problem_12.J for k in problem_12.K), sense=minimize)

# define constraints
problem_12.r1 = ConstraintList()
for j in problem_12.J:
    for k in problem_12.K:
        problem_12.r1.add(sum(problem_12.x[i,j,k] for i in problem_12.I) >= problem_12.DEM[j,k])
problem_12.r2 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r2.add(problem_12.x[i,4,k] >= problem_12.x[i,1,k])
problem_12.r3 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r3.add(problem_12.x[i,2,k] + problem_12.x[i,3,k] <= 1)
problem_12.r4 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r4.add(sum(problem_12.x[i,j,k] for j in range(1,4)) <= 2)
problem_12.r5 = ConstraintList()
for i in problem_12.I:
    for k in problem_12.K:
        problem_12.r5.add(sum(problem_12.x[i,j,k] for j in range(2,5)) <= 2)

solver = SolverFactory('gurobi')
results = solver.solve(problem_12)

print('Solución optima:')
for x in problem_12.x:
    print(x, value(problem_12.x[x]))
print('Valor óptimo:', value(problem_12.z))

Solución optima:
(1, 1, 1) -0.0
(1, 1, 2) -0.0
(1, 2, 1) 1.0
(1, 2, 2) -0.0
(1, 3, 1) 0.0
(1, 3, 2) 1.0
(1, 4, 1) 1.0
(1, 4, 2) -0.0
(2, 1, 1) 1.0
(2, 1, 2) 1.0
(2, 2, 1) 1.0
(2, 2, 2) -0.0
(2, 3, 1) -0.0
(2, 3, 2) 1.0
(2, 4, 1) 1.0
(2, 4, 2) 1.0
(3, 1, 1) 1.0
(3, 1, 2) 1.0
(3, 2, 1) 0.0
(3, 2, 2) 1.0
(3, 3, 1) 1.0
(3, 3, 2) 0.0
(3, 4, 1) 1.0
(3, 4, 2) 1.0
(4, 1, 1) -0.0
(4, 1, 2) -0.0
(4, 2, 1) -0.0
(4, 2, 2) -0.0
(4, 3, 1) -0.0
(4, 3, 2) -0.0
(4, 4, 1) -0.0
(4, 4, 2) -0.0
(5, 1, 1) 1.0
(5, 1, 2) 1.0
(5, 2, 1) 0.0
(5, 2, 2) 1.0
(5, 3, 1) 1.0
(5, 3, 2) 0.0
(5, 4, 1) 1.0
(5, 4, 2) 1.0
Valor óptimo: 780.0


# Problema 13

In [18]:
problem_13 = ConcreteModel()

# define sets
problem_13.I = RangeSet(1,4)
problem_13.J = RangeSet(1,3)

# define data
cost = {(1,1): 18, (1,2): 48, (1,3): 17,
        (2,1): 47, (2,2): 10, (2,3): 48,
        (3,1): 38, (3,2): 10, (3,3): 12,
        (4,1): 20, (4,2): 50, (4,3): 35}
fab_cap = {1: 75, 2: 75, 3: 75, 4: 75}
demand = {1: 100, 2: 120, 3: 80}

# define parameters
problem_13.c = Param(problem_13.I, problem_13.J, initialize=cost)
problem_13.of = Param(problem_13.I, initialize=fab_cap)
problem_13.dem = Param(problem_13.J, initialize=demand)

# define variables
problem_13.x = Var(problem_13.I, problem_13.J, within=NonNegativeReals)
problem_13.H = Var(problem_13.I, problem_13.I, within=NonNegativeReals)

# define objective function
problem_13.z = Objective(expr=sum(problem_13.c[i,j] * problem_13.x[i,j] for i in problem_13.I for j in problem_13.J), sense=minimize)

# define constraints
problem_13.r1 = ConstraintList()
for j in problem_13.J:
    problem_13.r1.add(sum(problem_13.x[i,j] for i in problem_13.I) >= problem_13.dem[j])
problem_13.r2 = ConstraintList()
for i in problem_13.I:
    problem_13.r2.add(sum(problem_13.x[i,j] for j in problem_13.J) <= sum(problem_13.H[k,i] for k in problem_13.I))
problem_13.r3 = ConstraintList()
for i in problem_13.I:
    problem_13.r3.add(sum(problem_13.H[i,k] for k in problem_13.I) <= problem_13.of[i])
problem_13.r4 = ConstraintList()
for i in problem_13.I:
    problem_13.r4.add(sum(problem_13.H[k,i] for k in problem_13.I) >= 10)

solver = SolverFactory('gurobi')
solver.solve(problem_13)

print('Solución optima:')
print('x[i,j] = ')
for i in problem_13.I:
    for j in problem_13.J:
        print('x[', i, ',', j, '] = ', value(problem_13.x[i,j]))
print('z = ', value(problem_13.z))

Solución optima:
x[i,j] = 
x[ 1 , 1 ] =  90.0
x[ 1 , 2 ] =  0.0
x[ 1 , 3 ] =  0.0
x[ 2 , 1 ] =  0.0
x[ 2 , 2 ] =  120.0
x[ 2 , 3 ] =  0.0
x[ 3 , 1 ] =  0.0
x[ 3 , 2 ] =  0.0
x[ 3 , 3 ] =  80.0
x[ 4 , 1 ] =  10.0
x[ 4 , 2 ] =  0.0
x[ 4 , 3 ] =  0.0
z =  3980.0


# Problema 14

In [19]:
problem_14 = ConcreteModel()

# define sets
problem_14.I = RangeSet(1,2)
problem_14.J = RangeSet(1,2)
problem_14.K = RangeSet(1,2)
problem_14.M = RangeSet(1,2)

# define data
sale_price_sw = {1: 200, 2: 150}
cost = {1: 25, 2: 15}
req_sw = {1: 1, 2: 2}
req_axe = {1: 2, 2: 1}
timeontasksw = {(1,1): 6, (1,2): 2, (2,1): 0, (2,2): 3}
timeontaskaxe = {1: 3, 2: 2}
reserve = {1: 600, 2: 500}

# define parameters
problem_14.Pe = Param(problem_14.I, initialize=sale_price_sw)
problem_14.Ph = Param(initialize=180)
problem_14.C = Param(problem_14.M, initialize=cost)
problem_14.Me = Param(problem_14.M, initialize=req_sw)
problem_14.Mh = Param(problem_14.M, initialize=req_axe)
problem_14.CapE = Param(initialize=200)
problem_14.CapH = Param(initialize=220)
problem_14.CapT = Param(initialize=25)
problem_14.Te = Param(problem_14.K, problem_14.J, initialize=timeontasksw)
problem_14.Th = Param(problem_14.K, initialize=timeontaskaxe)
problem_14.Horas = Param(initialize=480)
problem_14.Res = Param(problem_14.M, initialize=reserve)

# define variables
problem_14.E = Var(problem_14.I, problem_14.J, within=NonNegativeReals)
problem_14.H = Var(within=NonNegativeReals)

# define objective function
problem_14.z = Objective(expr=sum(problem_14.Pe[i] * problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + problem_14.Ph * problem_14.H - sum(problem_14.C[m] * (problem_14.Me[m] * sum(problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + problem_14.Mh[m] * problem_14.H) for m in problem_14.M), sense=maximize)

# define constraints
problem_14.r1 = Constraint(expr=(1/problem_14.CapE) * sum(problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + (1/problem_14.CapH) * problem_14.H <= 1)
problem_14.r2 = Constraint(expr=sum(problem_14.E[1,j] for j in problem_14.J) <= problem_14.CapT)
problem_14.r3 = ConstraintList()
for k in problem_14.K:
    problem_14.r3.add(sum(problem_14.Te[k,j] * problem_14.E[i,j] for j in problem_14.J for i in problem_14.I) + problem_14.Th[k] * problem_14.H <= problem_14.Horas)
problem_14.r4 = ConstraintList()
for m in problem_14.M:
    problem_14.r4.add(problem_14.Me[m] * sum(problem_14.E[i,j] for i in problem_14.I for j in problem_14.J) + problem_14.Mh[m] * problem_14.H <= problem_14.Res[m])

solver = SolverFactory('gurobi')
solver.solve(problem_14)

print('Solución optima:')
print ('H = ', value(problem_14.H))
print('E[i,j] = ')
for i in problem_14.I:
    for j in problem_14.J:
        print('E[', i, ',', j, '] = ', value(problem_14.E[i,j]))
print('z = ', value(problem_14.z))


Solución optima:
H =  96.0
E[i,j] = 
E[ 1 , 1 ] =  0.0
E[ 1 , 2 ] =  25.0
E[ 2 , 1 ] =  0.0
E[ 2 , 2 ] =  71.0
z =  21410.0


# Problema 15

In [20]:
problem_15 = ConcreteModel()

# define sets
problem_15.I = Set(initialize=['A','B','C','D','E','F','G','H','I'])
problem_15.J = Set(initialize=[1,2])
problem_15.Pred = Set(initialize=[('A','C'),('A','D'),('B','E'),('C','E'),('E','F'),('D','G'),('F','H'),('G','I')])

# define data
timeontask = {('A',1): 3, ('A',2):  1,
              ('B',1): 4, ('B',2):  2,
              ('C',1): 2, ('C',2):  0.5,
              ('D',1): 5, ('D',2):  2,
              ('E',1): 6, ('E',2):  1,
              ('F',1): 2, ('F',2):  1,
              ('G',1): 4, ('G',2):  3,
              ('H',1): 3, ('H',2):  2,
              ('I',1): 5, ('I',2): 4}
cost = {'A': 4,
        'B': 1,
        'C': 1,
        'D': 1,
        'E': 3,
        'F': 7,
        'G': 9,
        'H': 5,
        'I': 8}

# define parameters
problem_15.dueDate = Param(initialize=15)
problem_15.P = Param(problem_15.I, problem_15.J, initialize=timeontask)
problem_15.costo = Param(problem_15.I, initialize=cost)

# define variables
problem_15.x = Var(problem_15.I, within=NonNegativeIntegers)
problem_15.y = Var(problem_15.I, within=Binary)

# define objective function
problem_15.z = Objective(expr=sum(problem_15.costo[i] * problem_15.y[i] for i in problem_15.I), sense=minimize)

# define constraints
problem_15.r1 = ConstraintList()
for i in problem_15.I:
    problem_15.r1.add(problem_15.x[i] + problem_15.P[i,1] * (1-problem_15.y[i]) + problem_15.P[i,2] * problem_15.y[i] <= problem_15.dueDate)
problem_15.r2 = ConstraintList()
for i,k in problem_15.Pred:
    problem_15.r2.add(problem_15.x[k] >= problem_15.x[i] + problem_15.P[i,1] * (1-problem_15.y[i]) + problem_15.P[i,2] * problem_15.y[i])

solver = SolverFactory('gurobi')
solver.solve(problem_15)

print('Solución optima:')
for i in problem_15.I:
    print('x[', i, '] = ', value(problem_15.x[i]))
    print('y[', i, '] = ', value(problem_15.y[i]))
print('z = ', value(problem_15.z))


Solución optima:
x[ A ] =  -0.0
y[ A ] =  0.0
x[ B ] =  -0.0
y[ B ] =  0.0
x[ C ] =  3.0
y[ C ] =  1.0
x[ D ] =  3.0
y[ D ] =  1.0
x[ E ] =  4.0
y[ E ] =  0.0
x[ F ] =  10.0
y[ F ] =  0.0
x[ G ] =  6.0
y[ G ] =  0.0
x[ H ] =  12.0
y[ H ] =  0.0
x[ I ] =  10.0
y[ I ] =  0.0
z =  2.0
